<a href="https://colab.research.google.com/github/Pavitra-khare/DA6401_ASS_2B/blob/main/DL_ASS_2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 3628-pavitrakhare (3628-pavitrakhare-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
from sklearn.model_selection import train_test_split
from torchvision import models

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights

def modelStratPretrain(model_name, strategy):
    """
    Initialize a model with different pretrained layer freezing strategies:
    - strategy='freeze_all_except_last': Only the final FC layer is trainable
    - strategy='freeze_80_percent': Freeze 80% of the initial layers
    - strategy='freeze_fc_only': Only freeze the FC layer
    - strategy='train_from_scratch': Use untrained weights
    """
    def load_model(name, use_pretrained):
        if name == 'resnet':
            model_weights = ResNet50_Weights.DEFAULT if use_pretrained else None
            return resnet50(weights=model_weights)
        else:
            raise ValueError(f"Unsupported model: {name}")

    # Decide on using pretrained weights
    use_pretrained = (strategy != 'train_from_scratch')
    pretrained_model = load_model(model_name, use_pretrained)

    # Update the output layer for 10 classes
    num_features = pretrained_model.fc.in_features
    pretrained_model.fc = nn.Linear(num_features, 10)

    # Freezing strategies
    def freeze_except_fc(model):
        for name, param in model.named_parameters():
            if not name.startswith("fc"):
                param.requires_grad = False

    def freeze_fc_layer(model):
        for name, param in model.named_parameters():
            if name.startswith("fc"):
                param.requires_grad = False

    def freeze_initial_80_percent(model):
        children = list(model.named_children())
        freeze_upto = int(len(children) * 0.8)
        for idx, (_, module) in enumerate(children):
            if idx < freeze_upto:
                for param in module.parameters():
                    param.requires_grad = False

    if strategy == 'freeze_all_except_last':
        freeze_except_fc(pretrained_model)
    elif strategy == 'freeze_80_percent':
        freeze_initial_80_percent(pretrained_model)
    elif strategy == 'freeze_fc_only':
        freeze_fc_layer(pretrained_model)
    # No action for 'train_from_scratch' since all layers are already trainable

    return pretrained_model


In [ ]:
def loadTheData(data_dir, data_augumentation):
    """
    Load training and validation data from a directory with optional augmentation.
    Splits data into 80% training and 20% validation.
    """

    def get_transform(augment):
        if augment == 'Yes':
            return transforms.Compose([
                transforms.RandomResizedCrop(224),
                transforms.RandomHorizontalFlip(),
                transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
                transforms.RandomRotation(degrees=20),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
            ])
        return transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)
        ])

    # Apply transformations
    transform_pipeline = get_transform(data_augumentation)

    # Prepare dataset and split
    complete_dataset = ImageFolder(root=data_dir, transform=transform_pipeline)
    dataset_size = len(complete_dataset)
    indices = list(range(dataset_size))
    train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42)

    # Data samplers for randomized loading
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    # Initialize DataLoaders
    train_loader = DataLoader(
        complete_dataset,
        batch_size=32,
        sampler=train_sampler,
        num_workers=4,
        pin_memory=True
    )

    val_loader = DataLoader(
        complete_dataset,
        batch_size=32,
        sampler=val_sampler,
        num_workers=4,
        pin_memory=True
    )

    return train_loader, val_loader


In [ ]:
def trainDataTraining(model, train_data):
    """
    Trains the model on the provided training data for one epoch.
    Returns average loss and training accuracy.
    """
    loss_fn = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    model.train()

    total_loss = 0.0
    correct_preds = 0
    total_samples = 0

    for batch_inputs, batch_labels in train_data:
        batch_inputs, batch_labels = batch_inputs.to(device), batch_labels.to(device)

        optimizer.zero_grad()
        logits = model(batch_inputs)
        loss = loss_fn(logits, batch_labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        predictions = torch.argmax(logits, dim=1)
        correct_preds += (predictions == batch_labels).sum().item()
        total_samples += batch_labels.size(0)

    epoch_loss = total_loss / len(train_data)
    accuracy = (correct_preds / total_samples) * 100

    return epoch_loss, accuracy


In [ ]:
def validDataTesting(model, test_data):
    """
    Evaluates the model on the provided validation/test dataset.
    Returns the accuracy percentage.
    """
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for batch_x, batch_y in test_data:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            predictions = model(batch_x)
            predicted_labels = torch.argmax(predictions, dim=1)
            total_correct += (predicted_labels == batch_y).sum().item()
            total_samples += batch_y.size(0)

    accuracy_percent = (total_correct / total_samples) * 100
    return accuracy_percent


In [ ]:
def LoadTheTestData(test_dir, data_augumentation='No'):
    """
    Loads the test dataset with optional augmentation and returns a DataLoader.
    """
    # Choose transformation pipeline based on augmentation flag
    if data_augumentation == 'Yes':
        augmentation_pipeline = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
            transforms.RandomRotation(20),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])
    else:
        augmentation_pipeline = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])

    # Apply transformations and load dataset
    dataset = ImageFolder(root=test_dir, transform=augmentation_pipeline)
    loader = DataLoader(
        dataset,
        batch_size=32,
        shuffle=False,
        num_workers=4,
        pin_memory=True
    )
    return loader


In [ ]:
def trainCnnModel(model, train_data, val_data, test_data, epochs):
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    highest_val_acc = 0
    stop_limit = 2
    stop_counter = 0

    for ep in range(epochs):
        loss_avg, acc_train = trainDataTraining(model, train_data)

        print(f'Epoch {ep + 1}/{epochs} | Training Loss: {loss_avg:.4f} | Training Accuracy: {acc_train:.2f}%')
        wandb.log({'Train loss': loss_avg})
        wandb.log({'Train accuracy': acc_train})

        acc_val = validDataTesting(model, val_data)
        print(f'Epoch {ep + 1}/{epochs} | Validation Accuracy: {acc_val:.2f}%')
        wandb.log({'val_accuracy': acc_val})
        wandb.log({'epoch': ep})

        if acc_val > highest_val_acc:
            highest_val_acc = acc_val
            stop_counter = 0
        else:
            stop_counter += 1
            if stop_counter >= stop_limit:
                print(f"Early stopping at epoch {ep + 1}")
                break

    # Final evaluation on test data
    acc_test = validDataTesting(model, test_data)
    print(f'Test Accuracy: {acc_test:.2f}%')
    wandb.log({'test_accuracy': acc_test})

    print('Model training Completed.')


In [ ]:
# Update the sweep configuration to include the strategies
sweep_config = {
    'method': 'grid',  # Changed to grid to ensure all strategies are tested
    'metric': {
        'name': 'val_accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'model': {
            'values': ['resnet']
        },
        'strategy': {
            'values': [
                'freeze_all_except_last',  # Strategy 1
                'freeze_fc_only',          # Strategy 3
                'freeze_80_percent',       # Strategy 2
                'train_from_scratch'       # Strategy 4
            ]
        },
        'epoch': {
            'values': [10]
        }


    }
}

sweep_id = wandb.sweep(sweep_config, project='DL_ASS2_Prac')

In [ ]:
def main():
    with wandb.init() as run:
        # Dynamically create a run name based on current configuration
        config = wandb.config
        run_name_parts = [
            f"ep{config.epoch}",
            f"strategy-{config.strategy}",
            f"model-{config.model}"
        ]
        wandb.run.name = "_".join(run_name_parts)

        # Set up the model with the specified training strategy
        selected_model = modelStratPretrain(model_name=config.model, strategy=config.strategy)
        selected_model = selected_model.to(device)

        # Prepare training and validation datasets
        training_path = '/kaggle/input/my-dataset/inaturalist_12K/train'
        train_data, val_data = loadTheData(training_path, data_augumentation='No')

        # Prepare test dataset
        testing_path = '/kaggle/input/my-dataset/inaturalist_12K/val'
        test_data = LoadTheTestData(testing_path, data_augumentation='No')

        # Begin training and evaluate on test data
        trainCnnModel(selected_model, train_data, val_data, test_data, epochs=config.epoch)

wandb.agent(sweep_id, function=main, count=4)  # Execute with all 4 strategies
wandb.finish()


Create sweep with ID: yg114ke1
Sweep URL: https://wandb.ai/3628-pavitrakhare-indian-institute-of-technology-madras/DL_ASS2_B/sweeps/yg114ke1


wandb: Agent Starting Run: pntzdxvl with config:
wandb: 	epoch: 10
wandb: 	model: resnet
wandb: 	strategy: freeze_all_except_last


Epoch 1/10, Train Loss: 1.1234, Train Accuracy: 64.22%
Epoch 1/10, Validation Accuracy: 72.05%
Epoch 2/10, Train Loss: 0.8553, Train Accuracy: 71.73%
Epoch 2/10, Validation Accuracy: 72.75%
Epoch 3/10, Train Loss: 0.7922, Train Accuracy: 73.58%
Epoch 3/10, Validation Accuracy: 73.35%
Epoch 4/10, Train Loss: 0.7371, Train Accuracy: 75.41%
Epoch 4/10, Validation Accuracy: 75.10%
Epoch 5/10, Train Loss: 0.7495, Train Accuracy: 75.11%
Epoch 5/10, Validation Accuracy: 73.55%
Epoch 6/10, Train Loss: 0.7172, Train Accuracy: 75.92%
Epoch 6/10, Validation Accuracy: 73.90%
Epoch 7/10, Train Loss: 0.6923, Train Accuracy: 76.81%
Epoch 7/10, Validation Accuracy: 73.85%
Epoch 8/10, Train Loss: 0.6931, Train Accuracy: 76.90%
Epoch 8/10, Validation Accuracy: 73.85%
Epoch 9/10, Train Loss: 0.6600, Train Accuracy: 78.03%
Epoch 9/10, Validation Accuracy: 71.90%
Epoch 10/10, Train Loss: 0.6697, Train Accuracy: 77.63%
Epoch 10/10, Validation Accuracy: 75.40%
Test Accuracy: 76.45%
Training complete!


Train accuracy,▁▅▆▇▇▇▇▇██
Train loss,█▄▃▂▂▂▁▁▁▁
epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
val_accuracy,▁▃▄▇▄▅▅▅▁█
Train accuracy,77.6347
Train loss,0.66973
epoch,9
test_accuracy,76.45
val_accuracy,75.4


wandb: Agent Starting Run: fk1qcamn with config:
wandb: 	epoch: 10
wandb: 	model: resnet
wandb: 	strategy: freeze_80_percent


...

Train accuracy,▁▂▃▃▄▅▆▇▇█
Train loss,█▇▆▆▅▄▃▂▂▁
epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
val_accuracy,▁▃▄▆▇▇▇▇█▇
Train accuracy,84.37305
Train loss,0.4705
epoch,9
test_accuracy,46.85
val_accuracy,45.8


wandb: Agent Starting Run: g8svasgz with config:
wandb: 	epoch: 10
wandb: 	model: resnet
wandb: 	strategy: train_from_scratch


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Epoch 1/10, Train Loss: 2.3064, Train Accuracy: 19.74%
Epoch 1/10, Validation Accuracy: 20.15%
Epoch 2/10, Train Loss: 2.2084, Train Accuracy: 19.16%
Epoch 2/10, Validation Accuracy: 22.20%
Epoch 3/10, Train Loss: 2.1853, Train Accuracy: 20.55%
Epoch 3/10, Validation Accuracy: 20.80%
Epoch 4/10, Train Loss: 2.1428, Train Accuracy: 22.30%
Epoch 4/10, Validation Accuracy: 22.25%
Epoch 5/10, Train Loss: 2.1489, Train Accuracy: 22.23%
Epoch 5/10, Validation Accuracy: 23.30%
Epoch 6/10, Train Loss: 2.1172, Train Accuracy: 24.14%
Epoch 6/10, Validation Accuracy: 22.95%
Epoch 7/10, Train Loss: 2.0786, Train Accuracy: 25.50%
Epoch 7/10, Validation Accuracy: 23.85%
Epoch 8/10, Train Loss: 2.0630, Train Accuracy: 25.89%
Epoch 8/10, Validation Accuracy: 26.15%
Epoch 9/10, Train Loss: 2.0446, Train Accuracy: 26.42%
Epoch 9/10, Validation Accuracy: 25.40%
Epoch 10/10, Train Loss: 2.0132, Train Accuracy: 27.73%
Epoch 10/10, Validation Accuracy: 25.90%
Test Accuracy: 26.65%
Training complete!


Train accuracy,▁▁▂▄▄▅▆▆▇█
Train loss,█▆▅▄▄▃▃▂▂▁
epoch,▁▂▃▃▄▅▆▆▇█
test_accuracy,▁
val_accuracy,▁▃▂▃▅▄▅█▇█
Train accuracy,27.72847
Train loss,2.01322
epoch,9
test_accuracy,26.65
val_accuracy,25.9
